In [3]:
# pip install graphdatascience
from graphdatascience import GraphDataScience

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "Insert your password here - default password is neo4j"

gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), database="sna")

2.3.7


In [10]:
# List graphs in the catalog
list_result = gds.graph.list()
list_result

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,"{'p99': 3, 'min': 0, 'max': 3, 'mean': 0.8, 'p...",sna,sna,304 KiB,311592,5,4,{'relationshipProjection': {'VISITING': {'orie...,0.2,2023-06-01T13:02:47.383655200+02:00,2023-06-01T13:02:47.389590300+02:00,"{'graphProperties': {}, 'relationships': {'VIS...","{'graphProperties': {}, 'relationships': {'VIS..."


In [17]:
# Select relevant graph from the catalog
G = gds.graph.get("sna")

# Calculate and print degree centralities
degree_centralities = gds.degree.stream(G)
print(degree_centralities)

   nodeId  score
0       7    1.0
1       8    0.0
2       9    0.0
3      12    0.0
4      16    3.0
